<span style="color:blueviolet">**KODLISTOR_STOR: *k_id - kodlista - kod - svarstext***   
kanske går att hämta från min lista med dictionairies? 
  
  
<span style="color:blueviolet">**FREKVENSTABELL: *v_id - variabel - enkät - kod - frekvens***   
blir lite klurigare... jag måste göra något åt att det står "w+n" i enkätkolumnen. För att hitta rätt kod och frekvens borde jag kunna söka igenom den "högra delen" av filen, alltså där frekvenstabellerna ligger, och välja ut de celler som faktiskt har värden. FÖrmodligen står det NULL i de tomma cellerna efter att jag läst in excelfilen till en dataframe.  
  
    
    
<span style="color:blueviolet">**VARIABLER: *variabel - enkät - beskrivning - itemnr - enkättext - kodlista***   
borde inte vara så svår. Dela på på enkätkolumnen så att w+n separeras. Sedan måste item# hämtas från den högra delen av tabellen... det borde inte vara så klurigt.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Metadata.xlsx')

In [3]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

***  
# 1. Enkät  
  <span style="color:blueviolet">**Samla namnen på working-kolumner respektive nonworking-kolumner i två listor. Listorna kallas "WORKING" respektive "NONWORKING".**

In [4]:
WORKING = ['SLOSHW06', 'SLOSHW08', 'SLOSHW10', 'SLOSHW12', 'SLOSHW14', 'SLOSHW16', 'SLOSHW18']
NONWORKING = ['SLOSHNW06', 'SLOSHNW08', 'SLOSHNW10', 'SLOSHNW12', 'SLOSHNW14', 'SLOSHNW16', 'SLOSHNW18']

<span style="color:blueviolet">**Kolumnerna som står för vågor i SLOSH (alltså t.ex. SLOSHW06) innhåler NaN ifall en given variabel inte finns med i vågen. Om variabeln istället *finns* kommer vågkolumnen innehålla en string. De kolumner som heter "SLOSHW" står för enkäter för arbetande i SLOSH. Ifall kolumnen börjar med "SLOSHNW" står den för en enkät för icke-arbetande. Varje variabel kan förekomma i högst två kolumner (en SLOSHW och en SLOSHNW). Ifall en given variabel förekommer i *endast en* kolumn ska det i Enkät-kolumnen stå t.ex. "slosh work 06".**

In [5]:
for i,j in zip(WORKING, NONWORKING):
    df.loc[(df[i].isna() == False) & (df[j].isna() == True), ['Enkät']] = i
    df.loc[(df[j].isna() == False) & (df[i].isna() == True), ['Enkät']] = j

In [6]:
df['Enkät'].value_counts()

SLOSHW06       1210
SLOSHW08        889
w+n             748
SLOSHW10        669
SLOSHW12        579
SLOSHW14        567
SLOSHW16        517
SLOSHW18        426
SLOSHNW08       307
working         181
SLOSHNW12       160
SLOSHNW14       130
SLOSHNW16       106
SLOSHNW18        75
SLOSHNW10        41
non-working      34
SLOSHNW06        24
Name: Enkät, dtype: int64

<span style="color:blueviolet">**Frekvenstabellen visar att det finns tre värden i Enkät-kolumnen för vilka tillhörande variabler förekommer i två vågkolumner. Detta är oväntat. Alla variabler som förekommer i två vågor ska egentligen ha Enkät-värdet "w+n". Alltså har vi dokumenterat fel på något sätt? Tillsvidare får dessa variabler Enkät-värdet "oklar".**

In [7]:
df.loc[(df['Enkät'] == 'w+n') | (df['Enkät'] == 'working') | (df['Enkät'] == 'non-working'), ['Enkät']] = 'oklar'

  ***  
  # 2. Itemnr
  <span style="color:blueviolet"> **Nu är Enkät-kolumnen likadan som den i relationsdatabasen (förutom de som är "oklara"). Nu ska jag försöka skapa Itemnr-kolumnen**

In [8]:
df['Itemnr'] = np.nan

***
<span style="color:red">**TEST**

In [9]:
d = pd.DataFrame([[1,2,3], [4,5,6], [1,8,9]], columns = ['A', 'B', 'C'])
d['test'] = 0

In [10]:
d

,A,B,C,test
0,1,2,3,0
1,4,5,6,0
2,1,8,9,0


In [11]:
d.loc[d['A'] == 1, ['test']] = d['B']
d

,A,B,C,test
0,1,2,3,2
1,4,5,6,0
2,1,8,9,8


<span style="color:red">**TEST SLUT**  
***

In [12]:
for i,j in zip(WORKING, NONWORKING):
    df.loc[df['Enkät'] == i, ['Itemnr']] = df[i]
    df.loc[df['Enkät'] == j, ['Itemnr']] = df[j]

df.loc[df['Enkät'] == 'oklar', ['Itemnr']] = 'oklar'

<span style="color:blueviolet">**Nu tror jag att Itemnr-kolumnen är rätt. Testet ovan visar logiken bakon rad 12. När jag tittar i frekvenstabellen nedan ser man några konstiga värden längst ner. Det ser ut som att det någonstnas ligger en frekvenstabell på fel ställe...**

In [13]:
df.Itemnr.value_counts().head(5)

oklar    963
27        47
29        41
7_b       37
4         33
Name: Itemnr, dtype: int64

***  
# 3. Frekvens  
<span style="color:blueviolet"> **En frekvenstabell borde gå att göra på ungefär samma sätt som kolumnen för Itemnr.**


In [14]:
df['Frekvens'] = np.nan

In [15]:
WORKFREK = ['frek.1', 'frek.2', 'frek.3', 'frek.4', 'frek.5', 'frek.6', 'frek.7']
NONWORKFREK = ['frek.8', 'frek.9', 'frek.10', 'frek.11', 'frek.12', 'frek.13']

In [16]:
for i,j in zip(WORKFREK, NONWORKFREK):
    df.loc[(df[i].isna() == False) & (df[j].isna() == True), ['Frekvens']] = df[i]
    df.loc[(df[j].isna() == False) & (df[i].isna() == True), ['Frekvens']] = df[j]

***  
# 4. Kodlistor_stor  
<span style="color:blueviolet">**blablabalbal**

In [17]:
dfa = df[['Namn', 'Kodlista', 'Kod', 'Text']]

<span style="color:red">**TEST**

<span style="color:red">**TEST SLUT**

In [18]:
dfa.columns

Index(['Namn', 'Kodlista', 'Kod', 'Text'], dtype='object')

In [19]:
dfa.shape

(6664, 4)

In [20]:
kasta = []
behåll = []
for i in range(len(dfa['Kodlista'])):
    if dfa.iloc[i, 2] == 1:
        if dfa.iloc[i, 1] in behåll:
            kasta.append(dfa.iloc[i, 0])
        else:
            behåll.append(dfa.iloc[i, 1])        

In [21]:
for i in kasta:
    dfa.drop(dfa[dfa['Namn'] == i].index, inplace = True)

C:\Users\brafo\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
dfa.shape

(712, 4)

In [26]:
len(dfa['Kodlista'].unique())

116